In [1]:
import numpy as np
import matplotlib.pyplot as plt
import HUXt as H
import astropy.units as u
from skimage.segmentation import find_boundaries
import os
import glob
%matplotlib inline

def make_animation(src, dst, tidy=True):
    cmd = " ".join(["convert -delay 2.5 -loop 0 ", src, dst])
    os.system(cmd)
    if tidy:
        files = glob.glob(src)
        for f in files:
            os.remove(f)
    return

In [2]:
x = H.HUXt2D()

In [3]:
v_boundary = np.ones(128) * 400 * u.km/u.s
v_boundary[10:20] = v_boundary[10:20] * 2.0
v_boundary[60:80] = v_boundary[60:80] * 2.0
#v = x.solve_carrington_rotation(v_boundary)

In [4]:
#levels = np.arange(300, 850, 25)
#fig, ax = plt.subplots(figsize=(8,8),subplot_kw={"projection":"polar"})
#ax.contourf(x.phi.value, x.r.value, v.value, levels=levels, vmin=300, vmax=800)
#ax.set_ylim(0.0, x.r.value.max())

In [5]:
a = 10 * u.km
b = np.arange(1,20,1) * u.km
print(b > a)

[False False False False False False False False False False  True  True
  True  True  True  True  True  True  True]


In [6]:
time, v_amb, v_cme = x.solve_cone_cme(v_boundary)

In [7]:
for ii in range(v_cme.shape[0]):
        
    vc = v_cme[ii, :, :].copy()
    va = v_amb[ii, :, :].copy()
    
    phi = x.phi.value
    rad = x.r.value
    fig, ax = plt.subplots(figsize=(7,7), subplot_kw=dict(projection='polar'))
    ax.contourf(phi, rad, vc, levels=50, vmin=300, vmax=800)
    # Find CME boundary
    condition = (vc - va) > 20.0
    if np.any(condition):
        
        boundary = find_boundaries(condition, connectivity=1, mode="inner", background=0)
        points = np.argwhere(boundary)
        phi_b = phi[points[:, 1]]
        rad_b = rad[points[:, 0]]
        
        pb = np.unique(phi_b)
        r_lo = np.zeros(pb.shape)*np.NaN
        r_hi = np.zeros(pb.shape)*np.NaN
        for i, p in enumerate(pb):
            id_p = phi_b == p
            rad_sub = rad_b[id_p]

            id_above = rad_sub > np.mean(rad_b)
            if np.any(id_above):
                rad_hi = rad_sub[id_above]
                r_hi[i] = np.nanmax(rad_hi)

            id_below = rad_sub <= np.mean(rad_b)
            if np.any(id_below):
                rad_lo = rad_sub[id_below]
                r_lo[i] = np.nanmin(rad_lo)
        
        pb = np.hstack([pb, np.flipud(pb)])    
        rb = np.hstack([r_hi, np.flipud(r_lo)])    
        
        ax.plot(pb, rb, 'r-')
    
    ax.set_ylim(0, 215)
    ax.set_title("Simulation time: {:3.2f} days".format(time[ii]/86400))
        
    fig_dir = "C:\\Users\\yq904481\\PyCharmProjects\\HUXt\\figures"
    fig_name = "HUXt_t{:03d}.png".format(ii)
    fig_path = os.path.join(fig_dir,fig_name)
    fig.savefig(fig_path)
    plt.close('all')


C:\Anaconda3\lib\site-packages\matplotlib\projections\polar.py:55: RuntimeWarning: invalid value encountered in greater_equal
  r = np.where(r >= 0, r, np.nan)


In [8]:
src = "C:\\Users\\yq904481\\PyCharmProjects\\HUXt\\figures\\HUXt_t*.png"
dst = "C:\\Users\\yq904481\\PyCharmProjects\\HUXt\\figures\\HUXt_animation.gif"
make_animation(src, dst, tidy=True)